In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohammadjafarpoor","key":"41c25619380d36d715e20ff6a943b73e"}'}

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
 ! kaggle datasets list

In [ ]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset

In [ ]:
!unzip "/content/microsoft-catsvsdogs-dataset.zip" -d "/content/drive/MyDrive/Cat or Dog?"

## Import tools

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, experimental

In [ ]:
work_dir = "/content/drive/MyDrive/Cat or Dog?/PetImages"

In [ ]:
# Let's create function to filter & remove corrupted images
def CorruptImage():
    num_corrupted = 0
    for category in ("Cat", "Dog"):
        path = os.path.join(work_dir, category)

        for img in os.listdir(path):
            image_path = os.path.join(path, img)
            try:
                file_Obj = open(image_path, "rb")
                is_jfif = tf.compat.as_bytes("JFIF") in file_Obj.peek(10)
            finally:
                file_Obj.close()
            
            if not is_jfif:
                num_corrupted +=1
                os.remove(image_path)
    print("Total deleted images is = %d" % num_corrupted)

CorruptImage()

In [ ]:
# Let's function to remove non jpg images
def NonJpg():
    j=0
    for catagory in os.listdir(work_dir):
        sub_folder = os.path.join(work_dir, catagory)
        for name in os.listdir(sub_folder):
            img_path = os.path.join(sub_folder,name)
            if name.split('.')[1] == 'jpg':
                if os.path.getsize(img_path) <= 0:
                    if os.path.isfile(img_path) == False:
                        print(img_path)
                    os.remove(img_path)
                elif os.path.getsize(img_path) > 0:
                    j +=1
            if name.split('.')[1] != 'jpg':
                print(img_path)
                os.remove(img_path)
    print(j)
NonJpg()

In [ ]:
# Let's split image dataset into Train & validation

image_size = (128, 128)

train_dataset = preprocessing.image_dataset_from_directory(
    work_dir,
    image_size = image_size,
    validation_split= 0.2,
    subset="training",
    batch_size =32,    
    seed = 1337,
    shuffle = True,
)

test_dataset = preprocessing.image_dataset_from_directory(
    work_dir,
    image_size = image_size,
    validation_split = 0.2,
    subset = "validation",
    batch_size = 32,    
    seed = 1337,
    shuffle= True,
)

In [ ]:
input_shapes=(128, 128, 3)

model=Sequential()

model.add(experimental.preprocessing.Rescaling(1.0/255, input_shape=(input_shapes)))

model.add(Conv2D(16,(3,3),activation='relu',input_shape=input_shapes))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

model.fit(
    train_dataset, 
    validation_data=test_dataset,
    epochs=15
)

Epoch 1/15
586/586 [==============================] - 440s 747ms/step - loss: 0.6405 - accuracy: 0.6653 - val_loss: 0.7783 - val_accuracy: 0.5944
Epoch 2/15
289/586 [=============>................] - ETA: 3:24 - loss: 0.5092 - accuracy: 0.7472